In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from sklearn import datasets
from sklearn import metrics as m
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
from itertools import combinations
from sklearn import datasets
from sklearn import metrics as m
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.svm import SVC
import xgboost as xgb
from thundersvm import SVC as svmgpu
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder

randomseed = 10
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

In [3]:
def swapcolumns(trainval, testval, coldindexval):
    trainval[trainval != coldindexval] = 5
    testval[testval != coldindexval] = 5

    trainval[trainval == coldindexval] = 0
    trainval[trainval == 5] = 1

    testval[testval == coldindexval] = 0
    testval[testval == 5] = 1

    return trainval, testval

<IPython.core.display.Javascript object>

In [4]:
data = pd.read_csv("../dataset/seeds_dataset.txt", sep="\t", header=None)
data = shuffle(data)

le = LabelEncoder()
data.iloc[:, -1] = le.fit_transform(data.iloc[:, -1])
x = data.iloc[:, :-1]
y = data.iloc[:, -1]
print(np.unique(y))

[0 1 2]


<IPython.core.display.Javascript object>

In [5]:
# data = datasets.load_iris()
# x = data.data
# y = data.target.ravel()
# print(np.unique(y))

<IPython.core.display.Javascript object>

In [6]:
xtrain, xtest, ytrain_original, ytest_original = train_test_split(
    x, y, test_size=0.3, random_state=10
)

<IPython.core.display.Javascript object>

In [7]:
xgbmodel = xgb.XGBClassifier(random_state=randomseed)
xgbmodel.fit(xtrain, ytrain_original)
print(m.accuracy_score(ytest_original, xgbmodel.predict(xtest)))

0.9365079365079365


<IPython.core.display.Javascript object>

In [8]:
cvscore = cross_val_score(xgbmodel, x, y, cv=10)
print("mean= ", np.mean(cvscore), ", std= ", np.std(cvscore))

mean=  0.9333333333333333 , std=  0.03809523809523809


<IPython.core.display.Javascript object>

Confusion matrix based models
====

In [9]:
clf_c0 = []
acc_c0 = []
predictedprobablities_c0 = []
predictedconfusionmatrix_proba_c0 = []

<IPython.core.display.Javascript object>

# 1. baseline classifiers
=========

# 1.1 Class 0
====

In [10]:
# ===========================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 0)
# =================================================

rf_c0 = RandomForestClassifier(random_state=randomseed, n_estimators=50)
rf_c0.fit(xtrain, ytrain)
rfpred_c0 = rf_c0.predict(xtest)
print(m.accuracy_score(ytest, rfpred_c0))

clf_c0.append(rf_c0)
acc_c0.append(m.accuracy_score(ytest, rfpred_c0))
predictedprobablities_c0.append(rf_c0.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, rfpred_c0)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
predictedconfusionmatrix_proba_c0.append(propconfmat / 100)

0.9365079365079365


<IPython.core.display.Javascript object>

In [11]:
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 0)
# =================================================
xgbmodel_c0 = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
xgbmodel_c0.fit(xtrain, ytrain)
xgbmodelpred_c0 = xgbmodel_c0.predict(xtest)
print(m.accuracy_score(ytest, xgbmodelpred_c0))

clf_c0.append(xgbmodel_c0)
acc_c0.append(m.accuracy_score(ytest, xgbmodelpred_c0))
predictedprobablities_c0.append(xgbmodel_c0.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, xgbmodelpred_c0)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
predictedconfusionmatrix_proba_c0.append(propconfmat / 100)


0.9365079365079365


<IPython.core.display.Javascript object>

In [12]:
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 0)
# =================================================
svmmodel = svmgpu(random_state=randomseed, probability=True, C=1)
svmmodel.fit(xtrain, ytrain)
svmmodelpred = svmmodel.predict(xtest)
print(m.accuracy_score(ytest, svmmodelpred))

clf_c0.append(svmmodel)
acc_c0.append(m.accuracy_score(ytest, svmmodelpred))
predictedprobablities_c0.append(svmmodel.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, svmmodelpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
predictedconfusionmatrix_proba_c0.append(propconfmat / 100)

0.873015873015873


<IPython.core.display.Javascript object>

# 1.2 Class 1
====

In [13]:
clf_c1 = []
acc_c1 = []
predictedprobablities_c1 = []
predictedconfusionmatrix_proba_c1 = []

# =================================================
# classs 1
# =================================================
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 1)
# =================================================

rf = RandomForestClassifier(random_state=randomseed, n_estimators=50)
rf.fit(xtrain, ytrain)
rfpred = rf.predict(xtest)
print(m.accuracy_score(ytest, rfpred))

clf_c1.append(rf)
acc_c1.append(m.accuracy_score(ytest, rfpred))
predictedprobablities_c1.append(rf.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, rfpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
predictedconfusionmatrix_proba_c1.append(propconfmat / 100)


0.9682539682539683


<IPython.core.display.Javascript object>

In [14]:
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 1)
# =================================================
xgbmodel = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
xgbmodel.fit(xtrain, ytrain)
xgbmodelpred = xgbmodel.predict(xtest)
print(m.accuracy_score(ytest, xgbmodelpred))

clf_c1.append(xgbmodel)
acc_c1.append(m.accuracy_score(ytest, xgbmodelpred))
predictedprobablities_c1.append(xgbmodel.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, xgbmodelpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
predictedconfusionmatrix_proba_c1.append(propconfmat / 100)


1.0


<IPython.core.display.Javascript object>

In [15]:
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 1)
# =================================================
svmmodel = svmgpu(random_state=randomseed, probability=True, C=1)
svmmodel.fit(xtrain, ytrain)
svmmodelpred = svmmodel.predict(xtest)
print(m.accuracy_score(ytest, svmmodelpred))

clf_c1.append(svmmodel)
acc_c1.append(m.accuracy_score(ytest, svmmodelpred))
predictedprobablities_c1.append(svmmodel.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, svmmodelpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
predictedconfusionmatrix_proba_c1.append(propconfmat / 100)

0.9523809523809523


<IPython.core.display.Javascript object>

# 1.3 Class 2
====

In [16]:
#=================================================
# classs 2
#=================================================

clf_c2 = []
acc_c2 = []
predictedprobablities_c2 = []
predictedconfusionmatrix_proba_c2 = []

ytrain=ytrain_original.copy()
ytest=ytest_original.copy()
ytrain,ytest= swapcolumns(ytrain,ytest,2)

#=================================================

rf=RandomForestClassifier(random_state=randomseed, n_estimators=50)
rf.fit(xtrain,ytrain)
rfpred=rf.predict(xtest)
print(m.accuracy_score(ytest,rfpred))

clf_c2.append(rf)
acc_c2.append(m.accuracy_score(ytest,rfpred))
predictedprobablities_c2.append(rf.predict_proba(xtest))

confmat=m.confusion_matrix(ytest,rfpred)
confsumh=np.sum(confmat,axis=0)
propconfmat=confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:,i]= 100*propconfmat[:,i]/confsumh[i] 
predictedconfusionmatrix_proba_c2.append(propconfmat/100)


0.9047619047619048


<IPython.core.display.Javascript object>

In [17]:
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 2)
# =================================================
xgbmodel = xgb.XGBClassifier(random_state=randomseed, n_estimators=50)
xgbmodel.fit(xtrain, ytrain)
xgbmodelpred = xgbmodel.predict(xtest)
print(m.accuracy_score(ytest, xgbmodelpred))

clf_c2.append(xgbmodel)
acc_c2.append(m.accuracy_score(ytest, xgbmodelpred))
predictedprobablities_c2.append(xgbmodel.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, xgbmodelpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
predictedconfusionmatrix_proba_c2.append(propconfmat / 100)


0.9206349206349206


<IPython.core.display.Javascript object>

In [18]:
ytrain = ytrain_original.copy()
ytest = ytest_original.copy()
ytrain, ytest = swapcolumns(ytrain, ytest, 2)
# =================================================
svmmodel = svmgpu(random_state=randomseed, probability=True, C=1)
svmmodel.fit(xtrain, ytrain)
svmmodelpred = svmmodel.predict(xtest)
print(m.accuracy_score(ytest, svmmodelpred))

clf_c2.append(svmmodel)
acc_c2.append(m.accuracy_score(ytest, svmmodelpred))
predictedprobablities_c2.append(svmmodel.predict_proba(xtest))

confmat = m.confusion_matrix(ytest, svmmodelpred)
confsumh = np.sum(confmat, axis=0)
propconfmat = confmat.copy()
for i in range(propconfmat.shape[0]):
    propconfmat[:, i] = 100 * propconfmat[:, i] / confsumh[i]
predictedconfusionmatrix_proba_c2.append(propconfmat / 100)

0.9206349206349206


<IPython.core.display.Javascript object>

# 2. Combine all the base line models per class
=====

In [19]:
p_k1_c0 = 0
p_k1_c1 = 0
for i in range(len(predictedprobablities_c0)):
    p_k1_c0 += (
        predictedprobablities_c0[i][:, 0] * predictedconfusionmatrix_proba_c0[i][0][0]
        + predictedprobablities_c0[i][:, 1] * predictedconfusionmatrix_proba_c0[i][0][1]
    )

    p_k1_c1 += (
        predictedprobablities_c0[0][:, 0] * predictedconfusionmatrix_proba_c0[0][1][0]
        + predictedprobablities_c0[0][:, 1] * predictedconfusionmatrix_proba_c0[0][1][1]
    )

    print(i)
p_k1_c0 = p_k1_c0 / len(predictedprobablities_c0)
p_k1_c1 = p_k1_c1 / len(predictedprobablities_c0)

0
1
2


<IPython.core.display.Javascript object>

In [20]:
p_k2_c0 = 0
p_k2_c1 = 0
for i in range(len(predictedprobablities_c1)):
    p_k2_c0 += (
        predictedprobablities_c1[i][:, 0] * predictedconfusionmatrix_proba_c1[i][0][0]
        + predictedprobablities_c1[i][:, 1] * predictedconfusionmatrix_proba_c1[i][0][1]
    )

    p_k2_c1 += (
        predictedprobablities_c1[i][:, 0] * predictedconfusionmatrix_proba_c1[i][1][0]
        + predictedprobablities_c1[i][:, 1] * predictedconfusionmatrix_proba_c1[i][1][1]
    )

    print(i)
p_k2_c0 = p_k2_c0 / len(predictedprobablities_c0)
p_k2_c1 = p_k2_c1 / len(predictedprobablities_c0)

0
1
2


<IPython.core.display.Javascript object>

In [21]:
p_k3_c0 = 0
p_k3_c1 = 0
for i in range(len(predictedprobablities_c2)):
    p_k3_c0 += (
        predictedprobablities_c2[i][:, 0] * predictedconfusionmatrix_proba_c2[i][0][0]
        + predictedprobablities_c2[i][:, 1] * predictedconfusionmatrix_proba_c2[i][0][1]
    )

    p_k3_c1 += (
        predictedprobablities_c2[i][:, 0] * predictedconfusionmatrix_proba_c2[i][1][0]
        + predictedprobablities_c2[i][:, 1] * predictedconfusionmatrix_proba_c2[i][1][1]
    )

    print(i)
p_k3_c0 = p_k3_c0 / len(predictedprobablities_c2)
p_k3_c1 = p_k3_c1 / len(predictedprobablities_c2)

0
1
2


<IPython.core.display.Javascript object>

# 3. Calculate the probabilities of each class
====

In [22]:

# ci0 = ci1 = ci2 = 1 
 


p_k1_c0 = p_k1_c0 + (p_k2_c1 * p_k3_c1)


p_k2_c0 =p_k2_c0 + (p_k1_c1 * p_k3_c1)

p_k3_c0 = p_k3_c0 + (p_k2_c1 * p_k1_c1)



<IPython.core.display.Javascript object>

In [23]:
finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = p_k1_c0
finalcol[:, 1] = p_k2_c0
finalcol[:, 2] = p_k3_c0
finalpred = np.argmax(finalcol, axis=1)

ytest = ytest_original.copy()
print(m.accuracy_score(ytest, finalpred))
print(m.confusion_matrix(ytest, finalpred))

0.9365079365079365
[[18  0  4]
 [ 0 22  0]
 [ 0  0 19]]


<IPython.core.display.Javascript object>

In [24]:

finalcol = np.zeros((ytest.shape[0], 3))
finalcol[:, 0] = p_k1_c1
finalcol[:, 1] = p_k2_c1
finalcol[:, 2] = p_k3_c1
finalpred = np.argmin(finalcol, axis=1)

ytest = ytest_original.copy()
print(m.accuracy_score(ytest, finalpred))
print(m.confusion_matrix(ytest, finalpred))


0.9365079365079365
[[18  0  4]
 [ 0 22  0]
 [ 0  0 19]]


<IPython.core.display.Javascript object>

In [ ]:
0.9365079365079365